# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
# Load the data from part 1 and create a dataframe
cities = pd.read_csv("output_data/open_weathermap_data.csv", encoding="utf-8")
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Hilo,90,US,1606106761,88,19.73,-155.09,75.20,5.82
1,1,Port Elizabeth,40,ZA,1606106734,93,-33.92,25.57,62.60,3.36
2,2,Hermanus,95,ZA,1606106801,86,-34.42,19.23,60.01,5.70
3,3,Yarim,25,YE,1606106801,40,14.30,44.38,54.88,4.34
4,4,Port Macquarie,40,AU,1606106786,51,-31.43,152.92,84.20,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# define where to get the location information and humidity info
locations = cities[["Lat", "Lng"]]
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()

In [31]:
figure_layout = {
    'width': '300px',
    'height': '250px',
    'border': '5px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}


# Create the heatmap
fig = gmaps.figure(figure_layout, map_type='HYBRID')
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
# Narrow down the list of cities to those with ideal weather conditions
city_nar_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Wind Speed"] <10) & (cities["Cloudiness"] == 0), :]
city_nar_df = city_nar_df.dropna(how='any')
city_nar_df.reset_index(inplace=True)
del city_nar_df['index']
city_nar_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,7,Morón,0,AR,1606106724,60,-34.65,-58.62,73.00,5.01
1,28,Tomatlán,0,MX,1606106806,94,19.93,-105.25,70.97,2.13
2,125,Shirgaon,0,IN,1606106823,61,19.70,72.70,77.00,3.36
3,139,Manaquiri,0,BR,1606106826,88,-3.32,-60.35,78.80,4.70
4,207,Tura,0,IN,1606106769,49,25.52,90.22,73.60,3.80
5,269,Makkah al Mukarramah,0,SA,1606106851,66,21.43,39.83,74.44,3.15
6,329,Marsh Harbour,0,BS,1606106861,79,26.54,-77.06,76.50,9.35
7,364,Quelimane,0,MZ,1606106868,83,-17.88,36.89,78.80,6.93
8,398,Coahuayana Viejo,0,MX,1606106875,86,18.73,-103.68,74.39,2.82
9,417,Māngrol,0,IN,1606106938,37,21.12,70.12,79.03,3.53


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
# Create a list to store the returned hotels from the google API
hotellist = []

# Cycle through the narrowed city list dateframe
for i in range(len(city_nar_df)):
    lat = city_nar_df.loc[i]['Lat']
    lng = city_nar_df.loc[i]['Lng']
    
    # Google search parameters including location, distance, type and API key
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
city_nar_df["Hotel Name"] = hotellist
city_nar_df = city_nar_df.dropna(how='any')
city_nar_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,Morón,0,AR,1606106724,60,-34.65,-58.62,73.00,5.01,Hotel Morón
1,28,Tomatlán,0,MX,1606106806,94,19.93,-105.25,70.97,2.13,Norma franco
2,125,Shirgaon,0,IN,1606106823,61,19.70,72.70,77.00,3.36,LAXMI GARDEN BEACH RESORT
3,139,Manaquiri,0,BR,1606106826,88,-3.32,-60.35,78.80,4.70,
4,207,Tura,0,IN,1606106769,49,25.52,90.22,73.60,3.80,Hotel RIKMAN Continental


In [48]:
# Attempt to isolate the nulls and remove but after two rounds of cleaning they aren't comming out
#clean_city_df = city_nar_df.dropna(how='any', inplace=True)
#clean_city_df

# The above deleted the entire table...weird!

In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in city_nar_df.iterrows()]
locations = city_nar_df[["Lat", "Lng"]]

In [53]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))